# 4. GLMのモデル選択

Q. モデル選択とは

A. 
* 良い予測をするモデルを探すこと
* 「当てはまりのよさ」だけで選んではいけない
    * 複雑な統計モデルほど、観測データへの当てはまりは良くなる

Q. AICとは？

A.
* 「良い予測をするモデルが良いモデルである」という考えに基づいて設計された、モデル選択基準
* 「当てはまりの良さ重視」とは異なる考え方

## 4.1. データはひとつ、モデルはたくさん

* 第3章では、同じデータについて、いくつかの統計モデルで説明した
* 統計モデルのパラメータを最尤推定する際には、対数尤度が「いま手元にある観測データへの当てはまりの良さ」であると考え、これを最大にするようなパラメータの値を探した
* では、最大対数尤度こそがモデルの良さであると考えれば良いのだろうか？
* ->そうではない、というのが本章の要点

## 4.2. 統計モデルのあてはまりの悪さ：逸脱度

Q. 逸脱度とは？

A.
* あてはまりの良さである最大対数尤度を変形した統計量

Q. 

* ある対数尤度$ \log L( \{ \beta_j \} )$を$\log L$ と表記する
* この$\log L$を最大にするパラメータを探すのが最尤推定法
* 最大対数尤度を $\log L^*$と表記する


とした時、逸脱度の定義式は？

A. 
$$
D = -2\log L^*
$$
* 「あてはまりの良さ」ではなく「あてはまりの悪さ」を表現する指標
* 数値が大きいほど、あてはまりが悪い
* 最大対数尤度$\log L^*$に$-2$をかけている

Q. 残差逸脱度とは？

A.
* $D -$(ポアソン分布モデルで可能な最小逸脱度）
* この最小逸脱度をフルモデルと呼び、データ数と同じだけのパラメータを使って当てはめたモデル
* つまり残差逸脱度は、もっともあてはまりの良いフルモデルを基準とする「あてはまりの悪さ」の相対値。
* 残差逸脱度が最大になるのは、「もっともあてはまりの悪いモデル」の場合
* ポアソン分布だとこれは、もっともパラメーター数の少ないモデル null modelである。

### 体サイズ$x_i$だけに依存するモデルで計算する

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as scs
%matplotlib inline

In [2]:
data = pd.read_csv("data3a.csv")

In [3]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

/Users/takizawa/dev/python/books/midori/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [4]:
model = smf.glm('y ~ x',data=data,family=sm.families.Poisson())

In [5]:
result = model.fit()

In [6]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                            GLM   Df Residuals:                       98
Model Family:                 Poisson   Df Model:                            1
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -235.39
Date:                Thu, 15 Jun 2017   Deviance:                       84.993
Time:                        18:18:51   Pearson chi2:                     83.8
No. Iterations:                     4                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.2917      0.364      3.552      0.000       0.579       2.005
x              0.0757      0.036      2.125      0.034       0.006       0.145
==============================================================================
"""

* Pythonだと、Deviance 84.993と表示されている。おそらくResidual Deviance
* Null Devianceは表示されない

### full modelの逸脱度の計算

In [7]:
dpois = lambda x:scs.poisson.logpmf(x, x)

In [8]:
sum(dpois(data.y))

-192.8897525244958

In [9]:
sum(dpois(data.y)) * -2

385.77950504899161

* 最小逸脱度は385程度となった

### null modelの作成

In [10]:
model = smf.glm('y ~ 1',data=data,family=sm.families.Poisson())

In [11]:
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                  100
Model:                            GLM   Df Residuals:                       99
Model Family:                 Poisson   Df Model:                            0
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -237.64
Date:                Thu, 15 Jun 2017   Deviance:                       89.507
Time:                        18:18:51   Pearson chi2:                     87.1
No. Iterations:                     4                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.0580      0.036     57.586      0.000       1.988       2.128
==============================================================================
"""

* Devianceは89.507とResidualDevianceと一致している
* 一定モデルの最大対数尤度は-237.64となっており、逸脱度は475程度、最小逸脱度との差は475-385=90となり、Devianceと一致

## 4.3. モデル選択基準 AIC

* パラメータ数の多い統計モデルほど、データへのあてはまりが良くなる
* しかしそれは「たまたま得られたデータへの当てはめ向上を目的とする特殊化」であり、統計モデルの「予測の良さ」を損なっているのかもしれない

Q. モデル選択とは？

A.
* 複数の統計モデルの中から、何らかの基準で良いモデルを選択すること
* よく使われているモデル選択基準の1つがAIC(Akaike's information criterion)

Q. AICとは？

A.
* 予測の良さを重視するモデル選択基準
* $AIC = -2 \{ $ ( 最大対数尤度 ) - (最尤推定したパラメータ数)$\}$ 
$$= -2(\log L^* - k) $$
$$= D + 2k$$
* このAICが一番小さいモデルが良いモデルとなる

## 4.4. AICを説明するためのまた別の問題

* AICがなぜ統計モデルの予測力を表しているのか考える。

Q. ネストしているモデルとは？

A.

* 一定モデル($\log \lambda_i = \beta_1$) と xモデル（$\log \lambda_i = \beta_1 + \beta_2 x_i$）のように、一方のモデルに他方が含まれているようなくみあわせ

## 4.5. なぜAICでモデル選択して良いのか？

### 4.5.1. 統計モデルの予測の良さ：平均対数尤度

* 平均対数尤度は統計モデルの予測の良さを表す量

### 統計モデルのパラメータ推定の概念
* 人間には見えない真の統計モデルがある
* それを元に、観測データがサンプリングされる
* 観測データを元にパラメータ推定する
* そのため、パラメータ推定によって求められたパラメータと最大対数尤度は、たまたま得られた観測データ自身に対する当てはまりの良さであり、真の統計モデルに似ているかは考慮されない

Q. データ解析の狙いは？

A. 
* 観測される現象の背後にある「仕組み」の特定、もしくはそれを近似的に代替しうる統計モデルの構築
* そのため、「たまたま得られた」観測データへの当てはまりの良さを追求してはならない

Q. 推定された統計モデルが真の統計モデルにどれぐらい近いのかを調べる方法は？

A. 
* 真の統計モデルがわかっているのなら、そこから「予測の良さ評価用のデータ」を生成し、推定された一定モデルのどの程度当てはまるのか調べる
* 教科書の喩えだと、50個のサンプルを200セット作成。
* 各セットに対して対数尤度で評価する。その200の結果の平均を、平均対数尤度と呼ぶ。

### 4.5.2. 最大対数尤度のバイアス補正
* たまたま得られた観測データで推定されたモデルでは、当てはまりの良さが過大評価される（常に過大評価される訳ではない）
* 平均対数尤度の作成を12回繰り返す。すると、最大対数尤度（観測データへの当てはまり）と平均対数尤度（真の統計モデルから生成したデータへの当てはまり）は必ずしもどちらが良いとは言えない

Q. バイアスとは？

A. 
* $ b = \log L^* - E(\log L) $
* 「推定用データ生成、パラメータ推定、予測の良さを評価」を200回繰り返し、バイアスbの分布を描くことで、平均対数尤度より、最大対数尤度の方が平均的にどれぐらい大きいか(平均バイアス)、ということがわかる

Q. バイアス補正とは？

A.
* 平均的な$b$と最大対数尤度$\log L^*$の値を元に下記の計算式で平均対数尤度の推定量を得ること
* $E(\log L) = \log L^* - b$


Q. 最尤推定するパラメータを$k$個持つモデルの平均対数尤度の推定量を求める式は？


A.
* $E(\log L) = \log L^* - k$
* 上記に$-2$をかけたものが$AIC$。
* $AIC = -2 \times (\log L^* -1)$
* 平均対数尤度は「統計モデルの予測の良さ」であり、それに $-2$をかけた$AIC$は「予測の悪さ」と解釈できる。AICが低いモデルを選べば良い。

### 4.5.3. ネストしているGLM間のAIC比較

* AICによるモデル選択とは、複数のモデルのAICを比較してより小さいAICのモデルを選ぶこと
* バイアスのばらつきは大きいため、AICで評価して良いか不安になる。しかし同一データを使ってネストしている複数モデルのバイアス補正量のさの分布はそれほどばらつきの大きいものにはならないため、AICは有用である。
* ではネスとしていない複数のモデルで、AICによるモデル選択は可能だろうか。現時点では、「多分問題ないだろう」という理由で用いられている。

## 4.6. この章のまとめと参考文献

* 「当てはまりの良さ」最大対数尤度 $\log L^*$で「良い」モデルを選んではいけない。
* 「当てはまりの悪さ」逸脱度 $ D = -2 \log L^*$
* モデルは複雑化す流だけで、最大対数尤度 $\log L^*$は改善されてしまう。そのためモデルの複雑さを考慮したAICでモデル選択しなければならない。
* AICは平均対数尤度の推定値。これは最大対数尤度 $\log L^*$のバイアス補正によって評価される。
* AICは当てはまりの良いモデルではなく、良い予測をするモデルを選ぶ基準。

### AICによるモデル選択における注意点
* モデル選択の目的は「真の」モデルを求めることではない
* データ数が少ない場合には、「真の」モデルより、単純なモデルの方が予測能力が高い可能性もある
* データ数が少ない場合には、選択されたパラメータ数が過大であっても、「余計な」パラメータの推定地はゼロに近づくので問題ない
* モデル選択は、データ解析者が用意したモデルたちの中から良いモデルを選んでいるので、よりAICが良いモデルが他に存在する可能性がある。

* その他のモデル選択基準としては、交差検証法、ブートストラップ情報量基準など。